In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from models.vggish import VGGish
from utils import apply_melspectrogram_to_file
import math
import numpy as np
from models.model_keras_dropout import vgg_att
from dataloaders.DatagenVoxCeleb import get_keras_datagens

In [13]:
data_dir = '/media/data/bbekci/voxceleb2/data/dev/pkls/'
tr_txt = 'txts/tr_voxceleb_video_pkl_paths.txt'
val_txt = 'txts/val_voxceleb_video_pkl_paths.txt' 
batch_size = 32

In [14]:
tr_gen, val_gen = get_keras_datagens(data_dir, batch_size, split_by='video', split_size=0.3, txt_dirs=[tr_txt, val_txt])

In [11]:
for x,y in tr_gen:
    print(x.shape)
    print(y.shape)
    print(y)

 364 2552 4947 1090 2189 3961 4623
 4922 2577  964 2048 5547 1662 4686 3146 2605 2089 3819 5493 2437 1326
 5154  940 5694 3133]
(32, 300, 40)
(32,)
[3636 1114  831 4209  866  529 3704 2501 2403 2645 1307  386 4005 4568
 5172 5787 4859 4257  547 1116  268 1585  354 1716 5165 1408 5708 4017
 3690 4062 4107  602]
(32, 300, 40)
(32,)
[ 592  381 1795 1333 5038 3831 4154 5876 5845  168 5659 2375 4812 4068
 5806 1244 5672 2005 2416 3612 1101 4249 5884 3992 3385 4303 3103 5586
 4038 4148  139 5458]
(32, 300, 40)
(32,)
[1374 5100 5892 2967 1174 3046 3818 4641 5891 4187 1212 4310 4210 2796
  592 3003 4590 1918 2457 3856 3692 4718 3635 3314 5029 4960 1683   10
 5867 2898  826 5025]
(32, 300, 40)
(32,)
[5856 3013 3428 3755 3080 2667 5730    7 2841 3147 1519  793 3551 3098
  818  528  216  302  695 3115 1450  302 2475 4594 1595 5073  372 4836
  266 3012 2619 2905]
(32, 300, 40)
(32,)
[3064 4663 3212 5901 1248 3894  116 3284 4043  388 5904  162 2889 2444
 3755    2 1685 1095 4169 5486 5113 2235 5558

KeyboardInterrupt: 

In [15]:
sample_per_person = 300
batch_size = 10
num_class = 1251 #or 109
input_shape = (100, 40, 1)
persons_to_include = ['p228', 'p227', 'p225', 'p245', 'p247', 'p246', 'p228', 'p250', 'p251', 'p248', 'p231', 'p253']

In [16]:
"""tr_gen, val_gen, te_gen = get_datagen(sample_per_person, batch_size, apply_melspectrogram_to_file, include_person=None)"""

'tr_gen, val_gen, te_gen = get_datagen(sample_per_person, batch_size, apply_melspectrogram_to_file, include_person=None)'

In [17]:
"""for x, y in tr_gen:
    print(x.shape)
    print(y.shape)
    print(np.unique(np.argmax(y, axis=1)))
    break"""

'for x, y in tr_gen:\n    print(x.shape)\n    print(y.shape)\n    print(np.unique(np.argmax(y, axis=1)))\n    break'

In [18]:
model = vgg_att(num_class)

(None, 150, 20, 64)
(None, 75, 10, 128)
(None, 38, 5, 256)
(None, 19, 3, 512)
after reshape
(None, 19, 1536)
after attention
(None, 4, 1536)
after avgpool
(None, 1, 1536)
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 300, 40, 1)]      0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 40, 64)       640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 40, 64)       36928     
_________________________________________________________________
batch_normalization_4 (Batch (None, 300, 40, 64)       256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 150, 20, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D

In [19]:
opt = Adam(lr=1e-3)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
model.fit(tr_gen, validation_data=val_gen, epochs=20)

Epoch 1/20


ValueError: in user code:

    /home/bbekci/miniconda3/envs/inzpeech/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/bbekci/miniconda3/envs/inzpeech/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/bbekci/miniconda3/envs/inzpeech/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/bbekci/miniconda3/envs/inzpeech/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/bbekci/miniconda3/envs/inzpeech/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:532 train_step  **
        loss = self.compiled_loss(
    /home/bbekci/miniconda3